In [1]:
import pandas as pd
import os
import json
import uuid
from IPython.display import display_javascript, display_html, display
from copy import deepcopy

In [2]:
class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

In [3]:
form = json.load(open('./form.json',))

In [4]:
def get_schema_type(dtype):
    if dtype in ['SECTION','group']:
        dtype = 'object'
    if dtype in ['group']:
        dtype = 'array'
    if dtype in ['url','string','option']:
        dtype = 'string'
    return dtype

In [5]:
uiSchema = {}
enums = []

In [6]:
pd.DataFrame(enums)

""


In [7]:
def get_rules(s, enums, nm, schm, dependencies):
    oneOf = []
    rName = ""
    for g in list(filter(lambda en: en['estring'] in s['enum'] and en['id'] == s['id'], enums)):
        rName = g['name']
        rRule = {rName: {"enum": [g["enum"]]}}
        rRule.update({nm:schm})
        oneOf.append({"properties": rRule,"required":[nm]})
    if rName in dependencies:
        for dpd in dependencies[rName]['oneOf']:
            oneOf.append(dpd)
    return {rName: {"oneOf": oneOf}}

In [8]:
def dig_schema(pid,d, currentSchema, parentSchema, uiS):
    if d['type'] == 'SECTION':
        uiS.update({"classNames":"list-group"})
    elif d['type'] == 'group':
        uiS.update({"classNames":"list-group-item"})
    else:
        uiS.update({"classNames":"col-md-12"})
    if 'children' in d:
        dependencies = {}
        hasDependency = False
        for c in d['children']:
            name = ("{}_{}").format(pid,c['id'])
            uiS.update({name:{}})
            r = {'title':"{}. {}".format(c['id'], c['description']),
                 'type':get_schema_type(c['type'])}
            enum = []
            if 'enum' in c:
                enumNames = c['enum']
                for i, e in enumerate(c['enum']):
                    ename = c['id'] + '-' + str(i)
                    enum.append(ename)
                    enums.append({
                        'id':c['id'],
                        'name':name,
                        'question':c['description'],
                        'enum':ename,
                        'estring':e
                    })
                if r['type'] == 'string':
                    r.update({'enum':enum})
                    r.update({'enumNames':enumNames})
                    uiS.update({name:{"ui:widget": "radio"}})
                if r['type'] == 'array':
                    """
                    FOR CHECKBOXES
                    uiS.update({name:{"ui:widget": "checkboxes"}})
                    r.update({
                        "type": "array",
                        "items": {
                            "enum": enum,
                            "enumNames": enumNames,   
                        },
                        "uniqueItems": True
                    })
                    """
                    uiS.update({name:{"ui:widget": "radio"}})
                    r.update({
                        "type": "string",
                        "enum": enum,
                        "enumNames": enumNames,
                        "uniqueItems": True
                    })
            if 'children' in c:
                r.update({'properties':dig_schema(name,c,{}, r, uiS[name])})
            if 'rule' in c:
                hasDependency = True
                rule = get_rules(c['rule'], enums, name , r, dependencies)
                r.update({'hasRule':True})
                dependencies.update(rule)
            if 'rule' not in c:
                r.update({'hasRule':False})
            currentSchema.update({name: r})
        if hasDependency:
            parentSchema.update({'dependencies':dependencies})
    return currentSchema

In [9]:
schema = {}
for s in form:
    uiSchema.update({s['id']:{}})
    schema.update({
        s['id']: {
            'title':s['description'],
            'type':get_schema_type(s['type']),
            'properties': dig_schema(s['id'],s,{},{},uiSchema[s['id']])
        }
    })

In [10]:
RenderJSON(uiSchema)

In [11]:
schema = {'type':'object','properties':schema}
json_object = json.dumps(schema, indent = 4)
with open("./schema.json", "w") as outfile:
    outfile.write(json_object)

In [12]:
json_object = json.dumps(uiSchema, indent = 4)
with open("./uiSchema.json", "w") as outfile:
    outfile.write(json_object)